In [ ]:
import os, re

from math import ceil, floor, comb

In [ ]:
def cascade(n, i):
    a = []
    for k in range(i, 1, -1):
        m = 0
        while comb(m, k) < n:
            m += 1
        a.append(m - 1)
        n -= comb(m - 1, k)
    a.append(n)
    a.append(0)
    a.reverse()
    return a


def kruskal_katona(n, i, r):
    a = cascade(n, i)
    s = 1
    for k in range(i, r, -1):
        s += comb(a[k], k - r)
    return s

In [ ]:
def partitions(n, m, minimum=1):
    """
    Generator of partitions of n into m parts in lexicographic order with all parts at least minimum.
    Modification of Algorithm H from Knuth. http://www.cs.utsa.edu/~wagner/knuth/fasc3b.pdf
    Precondition: n >= m * minimum.
    """
    if m == 0:
        if n == 0:
            yield []
    elif m == 1:
        yield [n]
    elif m == 2:
        for i in range(minimum, floor(n / 2) + 1):
            yield [n - i, i]
    else:
        a = [n - (m - 1) * minimum]
        for i in range(m - 1):
            a.append(minimum)
        while True:
            yield a
            while a[1] < a[0] - 1:
                a[0] = a[0] - 1
                a[1] = a[1] + 1
                yield a
            j = 2
            s = a[0] + a[1] - 1
            while j < m and a[j] >= a[0] - 1:
                s += a[j]
                j += 1
            if j == m:
                return
            x = a[j] + 1
            a[j] = x
            j -= 1
            for i in range(j, 0, -1):
                a[i] = x
                s -= x
            a[0] = s

In [ ]:
def l(p):
    s = 0
    for x in p:
        s += f[x]
    return s

In [ ]:
def u(S):
    d = comb(n - r, r)
    return min(d * S, floor((d + comb(n - r - 1, r - 1)) * S * (comb(n, r) - S) / comb(n, r)))

K(9,4)

In [ ]:
n = 9
r = 4
d = 5

From Appendix 6.1, for $6\leq |\mathcal{C}|\leq 16$, we need to find lower bounds on $e(\mathcal{C},S)$ in order to define $f_3$. Note the cases of $1\leq |\mathcal{C}|\leq 5$ are already covered in Appendix 6.1.

The nauty executables should be in your PATH environment variable.

In [ ]:
# f = [None, 5, 8, None, None, None, 18, 21, 22, 25, 26, 27, 28, 29, 28, 31, 32]
f = [None, 5, 8, None, None, None]
for c in range(6, 17):
    f.append(max(int(e) for e in re.findall(r'\d+ graphs : e=(\d+)', os.popen('geng -CtfD5 ' + str(c) + ' | countg -g6: --e').read())))
for c in range(17, 116):
    f.append(ceil(2 * c * (126 - c) / 126))

In [ ]:
for c in range(5, 54):
    for S in range(c + 1, floor((n / r - 1) * c) + 1):
        ub = u(S) # upper bound
        # bounds on number of singletons a
        a_low = ceil((2 * c * (d - 1) - u(S)) / (d - 2))
        a_high = 1
        while a_high <= c and kruskal_katona(a_high, n - r, n - 2 * r) <= S:
            a_high += 1
        for a in range(a_low, a_high):
            # bounds on number of K_2s b
            b_low = max(0, ceil((6 * c - 126 + S - 5 * a) / 4))
            b_high = 1
            while a + b_high <= c and a + 2 * b_high <= 126 - S and kruskal_katona(a + b_high, n - r, n - 2 * r) <= S:
                b_high += 1
            for b in range(b_low, b_high):
                for p in partitions(126 - S - a - 2 * b, c - a - b, 6):
                    if l(p) + f[2] * b + f[1] * a <= ub:
                        print(p + [2] * b + [1] * a)

K(10,4)

In [ ]:
n = 10
r = 4
d = 15

Find lower bounds on $e(\mathcal{C},S)$.

In [ ]:
f = [None, 15, 28, None]
for c in range(4, 9):
    f.append(max(int(e) for e in re.findall(r'>C \d+ graphs with (\d+) edges', os.popen('geng -Ctv ' + str(c)).read())))
print(f)

However, $f_2$ must be non-decreasing.

In [ ]:
f = [None, 15, 28, None, 52, 63, 72, 72, 72]
for c in range(9, 95):
    f.append(ceil(9 * c * (210 - c) / 210))

Partition algorithm from Appendix 6.2.

In [ ]:
for c in range(48, 71):
    prev = []
    S = floor((n / r - 1) * c)
    ub = u(S)
    a_low = ceil((2 * c * (d - 1) - u(S)) / (d - 2))
    a_high = 1
    while a_high <= c and kruskal_katona(a_high, n - r, n - 2 * r) <= S:
        a_high += 1
    for a in range(a_low, a_high):
        b_low = max(0, ceil((4 * c - 210 + S - 3 * a) / 2))
        b_high = 1
        while a + b_high <= c and a + 2 * b_high <= 210 - S and kruskal_katona(a + b_high, n - r, n - 2 * r) <= S:
            b_high += 1
        for b in range(b_low, b_high):
            for p in partitions(210 - S - a - 2 * b, c - a - b, 4):
                if l(p) + f[2] * b + f[1] * a <= ub:
                    q = p + [2] * b + [1] * a
                    prev.append(q)
                    print(q)
    for S in range(floor((n / r - 1) * c) - 1, c, -1):
        ub = u(S)
        cur = []
        for p in prev:
            lb = l(p)
            if lb + f[p[0] + 1] - f[p[0]] <= ub:
                p[0] += 1
                cur.append(p[:])
                print(p)
                p[0] -= 1
            for i in range(1, c):
                if p[i - 1] >= p[i] + 1 and lb + f[p[i] + 1] - f[p[i]] <= ub:
                    p[i] += 1
                    cur.append(p[:])
                    print(p)
                    p[i] -= 1
        if (S - c) % 3 == 0 and 4 * c >= S:
            x = (4 * c - S) // 3
            y = (S - c) // 3
            if f[4] * y + f[1] * x <= ub and kruskal_katona(x, n - r, n - 2 * r) <= S:
                p = [4] * y + [1] * x
                cur.append(p)
                print(p)
        prev = cur
for c in range(71, 76):
    for S in range(c + 1, floor((n / r - 1) * c) + 1):
        ub = u(S)
        a_low = ceil((2 * c * (d - 1) - u(S)) / (d - 2))
        a_high = 1
        while a_high <= c and kruskal_katona(a_high, n - r, n - 2 * r) <= S:
            a_high += 1
        for a in range(a_low, a_high):
            b_low = max(0, ceil((4 * c - 210 + S - 3 * a) / 2))
            b_high = 1
            while a + b_high <= c and a + 2 * b_high <= 210 - S and kruskal_katona(a + b_high, n - r, n - 2 * r) <= S:
                b_high += 1
            for b in range(b_low, b_high):
                for p in partitions(210 - S - a - 2 * b, c - a - b, 4):
                    if l(p) + f[2] * b + f[1] * a <= ub:
                        q = p + [2] * b + [1] * a
                        prev.append(q)
                        print(q)